In [1]:
import pandas as pd
import numpy as np
from sklearn import *

In [2]:
df = pd.read_csv("/data/credit-default.csv")

In [3]:
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
checking_balance        1000 non-null object
months_loan_duration    1000 non-null int64
credit_history          1000 non-null object
purpose                 1000 non-null object
amount                  1000 non-null int64
savings_balance         1000 non-null object
employment_length       1000 non-null object
installment_rate        1000 non-null int64
personal_status         1000 non-null object
other_debtors           1000 non-null object
residence_history       1000 non-null int64
property                1000 non-null object
age                     1000 non-null int64
installment_plan        1000 non-null object
housing                 1000 non-null object
existing_credits        1000 non-null int64
default                 1000 non-null int64
dependents              1000 non-null int64
telephone               1000 non-null object
foreign_worker          1000 non-null object
jo

In [5]:
df.default.value_counts()

1    700
2    300
Name: default, dtype: int64

In [20]:
target = "default"
y = np.where(df[target] == 2, 1, 0)
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection\
.train_test_split(X.values, y
                , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=1
    #                                , include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", tree.DecisionTreeClassifier(max_depth = 6, min_samples_leaf= 10))
])

pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)
y_test_prob = pipe.predict_proba(X_test)[:, 1]

print("test accuracy", metrics.accuracy_score(y_test, y_test_pred))
print("train accuracy", metrics.accuracy_score(y_train, y_train_pred))
print("test precision", metrics.precision_score(y_test, y_test_pred))
print("train precision", metrics.precision_score(y_train, y_train_pred))
print("test recall", metrics.recall_score(y_test, y_test_pred))
print("train recall", metrics.recall_score(y_train, y_train_pred))

est = pipe.steps[-1][-1]

metrics.confusion_matrix(y_test, y_test_pred)

test accuracy 0.76
train accuracy 0.7757142857142857
test precision 0.6129032258064516
train precision 0.6838709677419355
test recall 0.4418604651162791
train recall 0.4953271028037383


array([[190,  24],
       [ 48,  38]])

In [21]:
from sklearn.tree import export_graphviz
export_graphviz(est, out_file = "tree.dot", feature_names = X.columns, filled=True)
!dot -Tpng tree.dot -o tree.png


In [19]:
y_test_prob

array([1.        , 1.        , 0.03529412, 0.43283582, 1.        ,
       0.09836066, 0.09836066, 0.39534884, 0.03529412, 0.3       ,
       0.6       , 0.43283582, 0.10526316, 0.03529412, 0.10526316,
       0.15151515, 0.03529412, 0.03529412, 0.        , 1.        ,
       0.75      , 0.39534884, 0.15151515, 0.39534884, 0.33333333,
       0.09836066, 1.        , 0.03529412, 0.09836066, 0.14285714,
       0.        , 0.14285714, 0.14285714, 0.09836066, 0.09836066,
       1.        , 0.10526316, 0.        , 0.09836066, 0.03529412,
       0.14285714, 0.39534884, 0.03529412, 0.43283582, 0.39534884,
       0.03529412, 0.09836066, 0.39534884, 0.03529412, 0.03529412,
       0.09836066, 0.39534884, 0.43283582, 0.43283582, 0.43283582,
       0.03529412, 0.39534884, 0.21428571, 0.03529412, 0.10526316,
       0.39534884, 0.03529412, 0.14285714, 0.03529412, 0.43283582,
       0.43283582, 0.15151515, 0.03529412, 0.43283582, 0.14285714,
       0.39534884, 0.03529412, 0.03529412, 0.39534884, 0.39534

In [28]:
summary = pd.DataFrame(dict(importance = est.feature_importances_, feature = X.columns))
summary.sort_values("importance", ascending = False)

,importance,feature
9,0.296347,checking_balance_unknown
0,0.156470,months_loan_duration
1,0.092540,amount
13,0.070180,credit_history_repaid
24,0.047968,savings_balance_< 100 DM
34,0.047706,other_debtors_guarantor
11,0.047576,credit_history_fully repaid
12,0.034449,credit_history_fully repaid this bank
8,0.033902,checking_balance_> 200 DM
39,0.033682,installment_plan_none
